<H1> Hello World!!! </H1>

<H2> Tasks - </H2>
<br>
- Read all text files <br>
- Convert into single csv <br>
- Upload to bigQuery Google <br>
- Visualizations with Tableau <br>
- Run unsupervised clustering <br>
- Deliver actual results <br>

In [12]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

In [2]:
#Read text files

OneFile = "GPS Test Data/LI5-1834230M/PASS/2018_03_08_GEDE-XTJ-53W.txt"

with open(OneFile) as f:
    print(f.read())

2018-03-08 19:40:56	Input SN : GEDE-XTJ-53W





GPS Module Detect....


Detect OK! Use time = 6031.637 sec.






Enable GPS Module....


at@gpsengine=1



OK

*********************

Enable GPS Module OK! Use time = 3.550 sec.






GPS Signal Test....


TTFF = 112.0 (s)



SNR1 = 34

SNR2 = 34

SNR3 = 34

SNR4 = 34

SNR5 = 38

SNR6 = 37

SNR7 = 30

SNR8 = 38

*********************

GPS Signal Test PASS! Use time = 121.054 sec.


Total Time 00:02:41



In [3]:
content = []
with open(OneFile) as f:
    content = f.readlines()
    content = [x.strip() for x in content] 
    content = [elem for elem in content if elem != ""]
    print(content) 

['2018-03-08 19:40:56\tInput SN : GEDE-XTJ-53W', '===============', 'GPS Module Detect....', '===============', 'Detect OK! Use time = 6031.637 sec.', '===============', 'Enable GPS Module....', '===============', 'at@gpsengine=1', 'OK', '*********************', 'Enable GPS Module OK! Use time = 3.550 sec.', '===============', 'GPS Signal Test....', '===============', 'TTFF = 112.0 (s)', 'SNR1 = 34', 'SNR2 = 34', 'SNR3 = 34', 'SNR4 = 34', 'SNR5 = 38', 'SNR6 = 37', 'SNR7 = 30', 'SNR8 = 38', '*********************', 'GPS Signal Test PASS! Use time = 121.054 sec.', 'Total Time 00:02:41']


In [4]:
def return_text_list(fname):
    content = []
    
    with open(fname) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
        content = [elem for elem in content if elem != ""]
        
    return content

In [19]:
def more_cleaning(folder_name,content):
    tmp_content = {}
    
    #Adding folder name
    tmp_content['Folder'] = folder_name
    
    #Extracting DateTime
    dT = content[0].split("\t")[0]
    
    #Should convert to datetime
    tmp_content['DateTime'] = dT
    
    #Extracting Input SN
    tmp_str = content[0].split("\t")[1]
    SN = tmp_str[tmp_str.find(":")+2:]
    
    tmp_content['SN'] = SN
    
    #Extracting other features
    for each in content:
        matchObj = re.match('^[^\=]*\=[^\=]*$',each)

        if matchObj:
            tmp = each.split("=")
            tmp_content[tmp[0].strip()] = tmp[1].strip()
        else:
            continue
    return tmp_content
    

In [9]:
a = " abc=de"
a.split("=")[0].strip()

'abc'

In [20]:
col_lst = ['Folder','DateTime','SN','UseTime','GPSEngine','WHERE TO APPLY','APPLICATION DEADLINE','SELECTION','NOTE','NOTICE']

#Read multiple files
base = "GPS Test Data/"

list_of_d = []
for folder_name in os.listdir(base):
    dir_name = base+folder_name+"/PASS/"
    
    for each2 in os.listdir(dir_name):
        tmp_content = return_text_list(dir_name+each2)
        
        row_dic = more_cleaning(folder_name,tmp_content)
        list_of_d.append(row_dic)
        

In [21]:
df = pd.DataFrame(list_of_d)

In [27]:
#Change type of data
df["SNR1"] = pd.to_numeric(df["SNR1"])
df["SNR2"] = pd.to_numeric(df["SNR2"])
df["SNR3"] = pd.to_numeric(df["SNR3"])
df["SNR4"] = pd.to_numeric(df["SNR4"])
df["SNR5"] = pd.to_numeric(df["SNR5"])
df["SNR6"] = pd.to_numeric(df["SNR6"])
df["SNR7"] = pd.to_numeric(df["SNR7"])
df["SNR8"] = pd.to_numeric(df["SNR8"])

df["at@gpsengine"] = pd.to_numeric(df["at@gpsengine"])

df['DateTime'] = pd.to_datetime(df['DateTime'])

df['TTFF'] = df['TTFF'].map(lambda x: x.rstrip(' (s)'))
df['Detect OK! Use time'] = df['Detect OK! Use time'].map(lambda x: x.rstrip(' sec.'))
df['Enable GPS Module OK! Use time'] = df['Enable GPS Module OK! Use time'].map(lambda x: x.rstrip(' sec.'))
df['GPS Signal Test PASS! Use time'] = df['GPS Signal Test PASS! Use time'].map(lambda x: x.rstrip(' sec.'))

In [28]:
df.head()

,Folder,DateTime,SN,Detect OK! Use time,at@gpsengine,Enable GPS Module OK! Use time,TTFF,SNR1,SNR2,SNR3,SNR4,SNR5,SNR6,SNR7,SNR8,GPS Signal Test PASS! Use time
0,LI5-1834230M,2018-03-08 19:40:56,GEDE-XTJ-53W,6031.637,1,3.550,112.0,34.0,34.0,34.0,34.0,38.0,37.0,30.0,38.0,121.054
1,LI5-1834230M,2018-03-08 20:25:17,GEPT-UCW-TUC,8847.955,1,0.610,70.0,29.0,33.0,34.0,33.0,37.0,27.0,37.0,27.0,121.050
2,LI5-1834230M,2018-03-08 20:36:15,GES6-4KZ-C93,9578.204,1,3.580,28.0,36.0,28.0,36.0,27.0,29.0,39.0,31.0,34.0,46.380
3,LI5-1834230M,2018-03-08 20:26:19,GES6-SZW-AFB,8755.451,1,0.610,120.0,37.0,37.0,36.0,37.0,30.0,34.0,28.0,34.0,120.402
4,LI5-1834230M,2018-03-08 20:27:00,GETA-PDD-W8K,9029.102,1,0.610,41.0,31.0,27.0,27.0,37.0,37.0,27.0,37.0,27.0,44.970


In [23]:
df.shape

(2180, 16)